In [106]:
!pip install statsmodels prophet pytorch-forecasting pandas matplotlib scikit-learn

# %%
# check torch gpu
import torch

device = None

print("<!> Selecting GPU as main device <!>")
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("<!> Using: " + torch.cuda.get_device_name(0) + "<!>")
else:
    print("<!> No GPU available <!>")
    exit(0)

# %%
import pandas as pd

print("<!> Loading test dataframe <!>")
df = pd.read_csv('/root/test/train_transactions.csv')

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from math import sqrt

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Trainer, QuantileLoss
from pytorch_forecasting.metrics import SMAPE, RMSE, MAPE

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

# Convert date column to datetime format
df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])

# Add time_idx
df['time_idx'] = range(len(df))

# Split data into train and validation sets
train_df = df[df['date'] < '2017-01-01']
validation_df = df[df['date'] >= '2017-01-01']

# Select one item (Example: Item ID 1)
item_id = '2'

# Add rolling averages as known covariates
for window_size in [7, 14, 30]:
    df[f'rolling_avg_{window_size}'] = df[item_id].rolling(window=window_size, min_periods=1).mean()

# Define training data
max_encoder_length = 30  # Adjust as needed
max_prediction_length = len(validation_df)  # Predict the length of the validation set

training = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target=item_id,
    group_ids=['item'],  # Use a single group ID since we're training on a single item
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long enough
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_ids={'item': 0},  # item ID as static feature
    time_varying_known_reals=[f'rolling_avg_{window_size}' for window_size in [7, 14, 30]],
    time_varying_unknown_reals=[item_id],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Define validation data
validation = TimeSeriesDataSet(
    validation_df,
    time_idx="time_idx",
    target=item_id,
    group_ids=['item'],  # Use a single group ID since we're training on a single item
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long enough
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_ids={'item': 0},  # item ID as static feature
    time_varying_known_reals=[f'rolling_avg_{window_size}' for window_size in [7, 14, 30]],
    time_varying_unknown_reals=[item_id],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Create dataloaders
batch_size = 64  # Adjust as needed
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

# Define model
pl.seed_everything(42)
trainer = Trainer(
    logger=TensorBoardLogger("lightning_logs"),
    callbacks=[EarlyStopping(monitor="val_loss", patience=10, mode="min"),
               LearningRateMonitor()],
    max_epochs=30,  # Adjust as needed
    accelerator="gpu",
    devices=1,
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # Adjust as needed
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,  # Adjust as needed
    output_size=1,  # Predict single value
    loss=QuantileLoss(),
    reduce_on_spline_basis=False
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

# Train model
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# Evaluate the model
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# Make predictions
actuals = torch.tensor([example[item_id] for example in validation], dtype=torch.float)
predictions = best_tft.predict(val_dataloader).flatten().cpu()

# Evaluate the model
def evaluate_forecast(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    smape = 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n=== Evaluation for TFT Model ===")
    print(f"  - Mean Absolute Percentage Error (MAPE): {mape:.2f}")
    print(f"  - Symmetric MAPE (SMAPE): {smape:.2f}")
    print(f"  - Mean Squared Error (MSE): {mse:.2f}")
    print(f"  - Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"  - R² Score: {r2:.2f}")

evaluate_forecast(validation_df[item_id], predictions)

# Plotting the results
plt.figure(figsize=(22, 5))
plt.plot(train_df['date'], train_df[item_id], label="Train", color="blue")
plt.plot(validation_df['date'], validation_df[item_id], label="Validation", color="green")
plt.plot(validation_df['date'], predictions, label="TFT Forecast", color="red", linestyle="dashed")
plt.axvline(train_df['date'].iloc[-1], color="black", linestyle="-.", label="Train/Validation Split")
plt.legend()
plt.title(f"TFT Forecast for Item {item_id}")
plt.show()

<!> Selecting GPU as main device <!>
<!> Using: NVIDIA GeForce GTX 1050<!>
<!> Loading test dataframe <!>


ImportError: cannot import name 'Trainer' from 'pytorch_forecasting' (/root/miniconda3/envs/ai/lib/python3.10/site-packages/pytorch_forecasting/__init__.py)